In [1]:
import sys, os
import pandas as pd
import numpy as np
sys.path.append('/'.join(os.getcwd().split('/')[:4]))
from helper import *
import glob 
from config.get import cfg
from tensorflow import keras

2022-01-09 19:56:28.308546: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-09 19:56:28.308584: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Loading data

In [2]:
X_train = np.load(cfg['files']["scaled_ae_train_features"])
X_test  = np.load(cfg['files']["raw_test_features"])
print(f"shapes : X_train={X_train.shape},X_test={X_test.shape}")

shapes : X_train=(5354, 3, 600, 2),X_test=(1340, 3, 600, 2)


# Loading models

In [3]:
model_paths = glob.glob(cfg["models"]["autoencoder"]+"*")
model_paths = [x for x in model_paths if not x.find(".")>0]
model_paths

['/home/kapps/dex-cyclic-arbitrage/models/embedding/autoencoder/results/CNN_fully_connected',
 '/home/kapps/dex-cyclic-arbitrage/models/embedding/autoencoder/results/fully_connected_3L',
 '/home/kapps/dex-cyclic-arbitrage/models/embedding/autoencoder/results/CNN_simpler_fully_connected']

In [9]:
models = {path.split("/")[-1] :  keras.models.load_model(path)  for path in model_paths[:2]}

# Models errors

In [10]:
def mean_squared_error(x,y):
    return np.sum((x-y)**2)/x.size

def eval_model(model):
    train_pred = model(X_train).numpy()
    test_pred  = model(X_test).numpy()
    return mean_squared_error(train_pred,X_train),mean_squared_error(test_pred,X_test)

In [11]:
test_mse  = []
train_mse = []
model_names = []

for model_name in models:
    print(model_name)
    train_e , test_e = eval_model(models[model_name])
    test_mse.append(test_e)
    train_mse.append(train_e)
    model_names.append(model_name)

CNN_fully_connected
fully_connected_3L


,model name,test_mse,train_mse
0,CNN_fully_connected,0.943945,0.622635
1,fully_connected_3L,0.462722,0.116216


# PCA errors

In [15]:
from sklearn import decomposition
N_train = len(X_train)
N_test  = len(X_test)
PCA_X_train = X_train.reshape(N_train,-1)
PCA_X_test = X_test.reshape(N_test,-1)
pca = decomposition.PCA(n_components=100)
pca.fit(PCA_X_train)
pca_train_mse = mean_squared_error(PCA_X_train, pca.inverse_transform(pca.transform(PCA_X_train)))
pca_test_mse =  mean_squared_error(PCA_X_test, pca.inverse_transform(pca.transform(PCA_X_test))) 


In [16]:
test_mse.append(pca_test_mse)
train_mse.append(pca_train_mse)
model_names.append("PCA")

# All results

In [17]:
pd.DataFrame({"model name":model_names , "test_mse" :test_mse , "train_mse": train_mse})

,model name,test_mse,train_mse
0,CNN_fully_connected,0.943945,0.622635
1,fully_connected_3L,0.462722,0.116216
2,PCA,0.232099,0.092100
